In [1]:
import math
import QuantLib as ql

if __name__ == "__main__":

    today = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = today

    spot = ql.QuoteHandle(ql.SimpleQuote(100))

    dc = ql.Actual365Fixed()
    qTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.025, dc))
    rTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, dc))

    vol_data = [
        # maturity in days, strike, volatility
        (30, 75, 0.13),
        (30, 100, 0.26),
        (30, 125, 0.3),
        (180, 80, 0.4),
        (180, 150, 0.6),
        (365, 110, 0.5)]

    calibration_set = ql.CalibrationSet(
        [(
            ql.VanillaOption(
                ql.PlainVanillaPayoff(ql.Option.Call, strike),
                ql.EuropeanExercise(today + ql.Period(maturity_in_days, ql.Days))
            ),
            ql.SimpleQuote(volatility)
        ) for maturity_in_days, strike, volatility in vol_data]
    )

    local_vol = ql.LocalVolTermStructureHandle(
        ql.AndreasenHugeLocalVolAdapter(
            ql.AndreasenHugeVolatilityInterpl(calibration_set, spot, rTS, qTS)
        )
    )

    option = calibration_set[-2][0]  # maturity in days: 180, strike: 150, vol: 0.6

    dummy_vol = ql.BlackVolTermStructureHandle()
    local_vol_process = ql.GeneralizedBlackScholesProcess(spot, qTS, rTS, dummy_vol, local_vol)

    option.setPricingEngine(ql.MCEuropeanEngine(
        local_vol_process, "lowdiscrepancy",
        timeSteps=100, brownianBridge=True, requiredSamples=32000, seed=42)
    )

    T = dc.yearFraction(today, option.exercise().lastDate())
    fwd = spot.value() * qTS.discount(T) / rTS.discount(T)
    vol = calibration_set[-2][1].value()

    expected = ql.BlackCalculator(
        ql.as_plain_vanilla_payoff(option.payoff()), fwd, vol * math.sqrt(T), rTS.discount(T)).value()

    print("Expected             : %.3f" % expected)
    print("Local Vol Monte-Carlo: %.3f" % option.NPV())

    time_steps = 100

    rsg = ql.GaussianRandomSequenceGenerator(
        ql.UniformRandomSequenceGenerator(time_steps, ql.UniformRandomGenerator(42))
    )

    path_generator = ql.GaussianPathGenerator(local_vol_process, 1.0, time_steps, rsg, False)
    next_path = path_generator.next().value()

    print("\nExample path: " + str({next_path.time(i): next_path.value(i) for i in range(len(next_path))}))

Expected             : 4.722
Local Vol Monte-Carlo: 4.721

Example path: {0.0: 100.0, 0.01: 99.01856110623987, 0.02: 101.15176906645377, 0.03: 105.35977909772349, 0.04: 102.38618771021574, 0.05: 104.02943784135306, 0.06: 106.25467550780667, 0.07: 107.0117602935268, 0.08: 107.75566347271368, 0.09: 104.65386015370001, 0.1: 103.76124482031071, 0.11: 98.47403302662325, 0.12: 92.84444168443274, 0.13: 86.49617906712255, 0.14: 86.04621243784463, 0.15: 90.22137203489719, 0.16: 88.51367370114949, 0.17: 89.42345078044406, 0.18: 85.4427534968789, 0.19: 87.3549614498285, 0.2: 88.7111512583252, 0.21: 81.5300705092172, 0.22: 76.47077505714033, 0.23: 82.72736665645127, 0.24: 84.63579317162208, 0.25: 87.88616680197912, 0.26: 93.39916030026896, 0.27: 90.4030494295544, 0.28: 79.69819271023898, 0.29: 76.97948723439183, 0.3: 84.67839772520487, 0.31: 81.75188431890943, 0.32: 82.63906124093843, 0.33: 81.02191847990984, 0.34: 81.84293381614617, 0.35000000000000003: 81.99586156071945, 0.36: 74.84044252783133,